# CAR-19 Data Analysis Goal 1: Confirm Pipeline Outputs

This notebook runs the code in the nircam_calib/comissioning/NRC_19_subarrays module in order to analyze data from CAR-19 (Subarray Verification)

## Table of Contents

* [Goal 1: Register and Combine data in pipeline](#goal_1)

In [ ]:
from glob import glob
import os

In [ ]:
from astropy.io import ascii, fits
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.pipeline.calwebb_image2 import Image2Pipeline
from jwst.pipeline.calwebb_image3 import Image3Pipeline
from jwst.pipeline.calwebb_tso3 import Tso3Pipeline
from jwst.source_catalog import SourceCatalogStep
import matplotlib.pyplot as plt
import numpy as np
from photutils.detection import DAOStarFinder

In [ ]:
from nircam_calib.commissioning.NRC_19_subarrays.subarray_loc_examination_prep import check_location
from nircam_calib.commissioning.NRC_19_subarrays import confirm_subarray_location_via_sources as locations
from nircam_calib.commissioning.NRC_19_subarrays import confirm_telescope_pointing as pointing
from nircam_calib.commissioning.NRC_19_subarrays import confirm_count_rates as count_rates
from nircam_calib.commissioning.utils.display import overlay_catalog

In [ ]:
import jwst
print(jwst.__version__)

In [ ]:
base_dir = '/ifs/jwst/wit/nircam/simulationWG/Imaging/CAR-19/2021_June'
#base_dir = 'path_to_simulationWG_CAR-19_directory'

<a id='goal_1'></a>
## Goal 1: Check that calibration pipeline correctly registers and combines subarray data

MAST will run the pipeline with default parameters, which should work for our analysis here. The only case we may have to run manually is Stage 3 when trying to combine data from multiple subarrays. I don't think MAST will do this by default. But it would be worth testing here to make sure it's done correctly.

In [ ]:
sw_imaging_mult_subarrays_lvl3_asn = 'Pipeline_Level3/level3_all_sw_subarrays_asn.json'
lw_imaging_mult_subarrays_lvl3_asn = 'Pipeline_Level3/level3_all_lw_subarrays_asn.json'

In [ ]:
result = Image3Pipeline()
result.source_catalog.save_results = True
result.save_results = True
output_dir = os.path.join(base_dir, 'Pipeline_Level3/')
result.output_dir = output_dir
result.run(sw_imaging_mult_subarrays_lvl3_asn)

In [ ]:
result = Image3Pipeline()
result.save_results = True
result.source_catalog.save_results = True
output_dir = os.path.join(base_dir, 'Pipeline_Level3/')
result.output_dir = output_dir
result.run(lw_imaging_mult_subarrays_lvl3_asn)

<b>Examine output:</b>  
      measure FWHM of sources  
      visual inspection of mosaic size/shape

### Examine the Results

In [ ]:
def display(files, titles, minval=0, maxval=10):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 10))

    for ax, filename, title in zip(axes.flat, files, titles):
        img = fits.getdata(filename)
        ax.set_axis_off()
        im = ax.imshow(img, origin='lower', cmap='viridis', vmin=minval, vmax=maxval)
        ax.set_title(title)

    cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.5)
    cbar.set_ticks(np.linspace(minval, maxval, 5))
    plt.show()

In [ ]:
all_subarrays_combined_files = sorted(glob(os.path.join(base_dir, 'Pipeline_Level3/all_subarrays_*_i2d.fits')))
full_combined_files = sorted(glob(os.path.join(base_dir, 'Pipeline_Level3/full_?w_i2d.fits')))
sub160_combined_files = sorted(glob(os.path.join(base_dir, 'Pipeline_Level3/sub160_?w_i2d.fits')))
sub320_combined_files = sorted(glob(os.path.join(base_dir, 'Pipeline_Level3/sub320_?w_i2d.fits')))
sub640_combined_files = sorted(glob(os.path.join(base_dir, 'Pipeline_Level3/sub640_?w_i2d.fits')))

sub400p_cal_files = [os.path.join(base_dir, 'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits'),
                     os.path.join(base_dir, 'Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits')]
sub64p_cal_files = [os.path.join(base_dir, 'Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits'),
                    os.path.join(base_dir, 'Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits')]

In [ ]:
sub640_combined_files

In [ ]:
# TAKE A CAREFUL LOOK AT THIS ONE. Combining data from different apertures
# has been a problem in the pipline, with poor alignment between apertures.
# This shows up as blurry sources in the areas where multiple apertures overlap.
# This blurriness is most likely due to Tweakreg screwing things up, so if things
# are blurry, try rerunning the pipeline without tweakreg.
lwsw_titles = ['LW All Apertures', 'SW All Apertures']
display(all_subarrays_combined_files, lwsw_titles)

In [ ]:
lwsw_titles = ['LW Full Frame', 'SW Full Frame']
display(full_combined_files, lwsw_titles)

In [ ]:
lwsw_titles = ['LW SUB160', 'SW SUB160']
display(sub160_combined_files, lwsw_titles)

In [ ]:
lwsw_titles = ['LW SUB320', 'SW SUB320']
display(sub320_combined_files, lwsw_titles)

In [ ]:
lwsw_titles = ['LW SUB640', 'SW SUB640']
display(sub640_combined_files, lwsw_titles)

<img src='/Users/hilbert/Desktop/NRC-19_SUB400P_fov.png)'>Image description</img>

In [ ]:
lwsw_titles = ['LW SUB400P', 'SW SUB400P']
display(sub400p_cal_files, lwsw_titles, minval=0, maxval=1000)

<img src='/Users/hilbert/Desktop/NRC-19_SUB64P_fov.png)'>Image description</img>

In [ ]:
lwsw_titles = ['LW SUB64P', 'SW SUB64P']
display(sub64p_cal_files, lwsw_titles, minval=0, maxval=1000)

## Try measuring FWHM on stars in the combined data

In [ ]:
def fwhm_stats(data, fwhm=3.0, sigma=10):
    """Calculate FWHM stats on the sources in the input image"""
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=sigma*std)
    sources = daofind(data - median)
    for col in sources.colnames:  
        sources[col].info.format = '%.8g'  # for consistent table output
    print(sources)